In [ ]:
!pip install mplfinance
!pip install yfinance
!pip install pandas_ta

In [ ]:
from datetime import timedelta, date
import pandas as pd
import shutil
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import mplfinance as mf
import yfinance as yf
import pandas_ta as ta

from mplfinance.original_flavor import candlestick_ohlc
from matplotlib.pylab import date2num
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta


import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Flatten, Dense, Dropout, LSTMCell
%matplotlib inline

from keras import optimizers
from keras.layers import Dropout, Flatten, Dense, Activation, BatchNormalization
from keras.layers.convolutional import Convolution2D, MaxPooling2D, Conv2D
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

In [ ]:
path = '/kaggle/working/'
shutil.os.mkdir(path + "model")
shutil.os.mkdir(path + "predict")
shutil.os.mkdir(path + "Candle_Data")
shutil.os.mkdir(path + "Candle_Data/Up")
shutil.os.mkdir(path + "Candle_Data/Down")

In [ ]:
# Create your own Custom Strategy
CustomStrategy = ta.Strategy(
    name="Momo and Volatility",
    description="ma10, ma30, rsi, macd",
    ta=[
        {"kind": "sma", "length": 10},
        {"kind": "sma", "length": 30},
        {"kind": "rsi"},
        {"kind": "macd", "fast": 8, "slow": 21},
    ]
)

In [ ]:
def plot_chart(df, path):
    
    #set date as index
    data = df.copy()
    data.index=data['Date']
    data.drop(["Date"], axis=1, inplace=True)


    # Create figure and set axes for subplots
    fig = plt.figure()
    fig.set_size_inches((20, 16))
    ax_candle = fig.add_axes((0, 0.72, 1, 0.32))
    ax_macd = fig.add_axes((0, 0.48, 1, 0.2), sharex=ax_candle)
    ax_rsi = fig.add_axes((0, 0.24, 1, 0.2), sharex=ax_candle)
    ax_vol = fig.add_axes((0, 0, 1, 0.2), sharex=ax_candle)
    
    # Format x-axis ticks as dates
    ax_candle.xaxis_date()
    
    # Get nested list of date, open, high, low and close prices
    ohlc = []
    for date, row in data.iterrows():
        openp, highp, lowp, closep = row[:4]
        ohlc.append([date2num(date), openp, highp, lowp, closep])
 
    # Plot candlestick chart
    ax_candle.plot(data.index, data["ma10"], label="MA10")
    ax_candle.plot(data.index, data["ma30"], label="MA30")
    candlestick_ohlc(ax_candle, ohlc, colorup="g", colordown="r", width=0.8)
    
    # Plot MACD
    ax_macd.plot(data.index, data["macd"], label="macd")
    ax_macd.bar(data.index, data["macd_hist"] * 3, label="hist")
    ax_macd.plot(data.index, data["macd_signal"], label="signal")
    
    # Plot RSI
    # Above 70% = overbought, below 30% = oversold
    #ax_rsi.set_ylabel("(%)")
    ax_rsi.plot(data.index, [70] * len(data.index), label="overbought")
    ax_rsi.plot(data.index, [30] * len(data.index), label="oversold")
    ax_rsi.plot(data.index, data["rsi"], label="rsi")
    
    # Show volume in millions
    ax_vol.bar(data.index, data["Volume"] )
    #ax_vol.set_ylabel("(Million)")
   
    # Save the chart as PNG
    ax_candle.axes.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
    ax_macd.axes.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
    ax_rsi.axes.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
    ax_vol.axes.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))

    fig.savefig(path, bbox_inches="tight")
    #plt.show()
    plt.close(fig)

In [ ]:
#read in data
data = pd.read_csv('/kaggle/input/5years-dailystock-quotes/original.csv', delimiter=',')
data = data[['open_price','high_price','low_price','close_price','volume','symbol','datetime']]
data.dataframeName = 'original.csv'
# Convert date to timestamp and make index
new_format = "%Y-%m-%dT%H:%M:%SZ"
data['date'] = pd.to_datetime(data['datetime'], format=new_format)
data.drop(["datetime"], axis=1, inplace=True)

nRow, nCol = data.shape
print(f'There are {nRow} rows and {nCol} columns')

In [ ]:
#generate plots
tick=['FBNC', 'BANF', 'SLCT']#, 'SCHW', 'SBCF', 'JCOM', 'ALLY', 'PB', 'CBFV', 'OSK', 'SKYW']
n_days=2
window = 90
fraction_movement=0.037

for company in tick:
    print(company)
    ticker = data['symbol']==company
    df = data[ticker].loc[data.first_valid_index():]
    df.set_index(pd.DatetimeIndex(df["date"]), inplace=True)
    df.ta.strategy(CustomStrategy)
    df.columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'Symbol', 'Date',
                  'ma10', 'ma30', 'rsi', 'macd', 'macd_hist','macd_signal']
    df = df.dropna()
    df=df.reset_index(drop=True)
    df['Trend'] = None
    
    for i in range(len(df)):
        try :
            for n in range(n_days): 
                if  df.loc[i,'Close'] - df.loc[i+n,'Close'] >= fraction_movement*df.loc[i,'Close']:
                    df.loc[i,'Trend'] ='Down'
                    
                    if i > window :
                        plot_chart(df[i-window:i], '/kaggle/working/Candle_Data/Down/{0}{1}.png'.format(df['Symbol'][i],i))
                    #print('Down',i,n)
                    break
                    
                elif df.loc[i+n,'Close'] - df.loc[i,'Close'] >= fraction_movement*df.loc[i,'Close'] :
                    df.loc[i,'Trend']='Up'
                    
                    if i > window :    
                        plot_chart(df[i-window:i], '/kaggle/working/Candle_Data/Up/{0}{1}.png'.format(df['Symbol'][i],i))
                    #print('Up',i,n)
                    break
                    
                else :
                    df.loc[i,'Trend']= 'No Trend'
        except :
            #print(i)
            pass

In [ ]:
from PIL import Image
#Image.open('/kaggle/working/Candle_Data/Up/{0}{1}.png'.format(df['Symbol'][i],i))

In [ ]:
# Input the size of your sample images
img_width, img_height = 800, 575
# Enter the number of samples, training + validation
#nb_train_samples = 13204
#nb_validation_samples = 1412
nb_filters1 = 16
nb_filters2 = 32
nb_filters3 = 64
conv1_size = 4
conv2_size = 2
conv3_size = 6
pool_size = 2
# We have 2 classes, up and down
classes_num = 2
batch_size = 64
lr = 0.002
chanDim =3

In [ ]:
#build model

model = tf.keras.Sequential()
model.add(Convolution2D(nb_filters1, conv1_size, conv1_size, padding ='same', input_shape=(img_height, img_width , 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

model.add(Convolution2D(nb_filters2, conv2_size, conv2_size, padding ="same"))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size), data_format="channels_last"))

model.add(Convolution2D(nb_filters3, conv3_size, conv3_size, padding ='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size), data_format="channels_last"))

model.add(Flatten())
model.add(Dense(1024 * 2))
model.add(Dense(1024))
model.add(Dense(1024 / 2))

model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(classes_num, activation='softmax'))

model.summary()
model.compile(loss='categorical_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])

In [ ]:
#create generators

validation_split = 0.2
data_dir = '/kaggle/working/Candle_Data/'

train_datagen = ImageDataGenerator(
    validation_split=validation_split, 
    rescale=1./255,
    rotation_range=40,
    zoom_range=0.02,
)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    shuffle=True,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=True,
    class_mode='categorical',
    subset='validation'
)

In [ ]:
#create callbacks

"""
Tensorboard log
"""
target_dir = "/kaggle/working/model/weights.hdf5"
#if not os.path.exists(target_dir):
#  os.mkdir(target_dir)
#model.save('./src/models/model.h5')
#model.save_weights('./src/models/weights.h5')

checkpoint = ModelCheckpoint(target_dir, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]


In [ ]:
#build model

epochs = 10
model.fit_generator(
    train_generator,
    #steps_per_epoch=nb_train_samples//batch_size,
    epochs=epochs,
    shuffle=True,
    validation_data=validation_generator,
    callbacks=callbacks_list,
    #validation_steps=nb_validation_samples//batch_size
)

In [ ]:
#need to create heatmap

In [ ]:
#set parameters for data pull

today = date.today()
end = today.strftime("%Y-%m-%d")
print(end)
endDate = date.today() - timedelta(days=180)
start = endDate.strftime("%Y-%m-%d")
print(start)

In [ ]:
#lets grab some data and predict!

df_yahoo = yf.download('FBNC',
                        start=start,
                        end=end,
                        progress=False)

df_yahoo.drop(["Adj Close"], axis=1, inplace=True)
df_yahoo=df_yahoo.reset_index()
df_yahoo['Symbol'] = 'FBNC'
df_yahoo = df_yahoo[['Open', 'High', 'Low', 'Close', 'Volume', 'Symbol', 'Date']]
df_yahoo['Symbol'] = 'FBNC'

df_yahoo

In [ ]:
#do TA

df_yahoo.ta.strategy(CustomStrategy)
df_yahoo.columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'Symbol', 'Date',
                    'ma10', 'ma30', 'rsi', 'macd', 'macd_hist','macd_signal']
df_yahoo

In [ ]:
#grab last window
df_yahoo = df_yahoo.tail(window)
df_yahoo

In [ ]:
#take a look

plot_chart(df_yahoo, '/kaggle/working/predict/tomorrows_prediction.png')
Image.open('/kaggle/working/predict/tomorrows_prediction.png')

In [ ]:
def predict():
    
    x = load_img('/kaggle/working/predict/tomorrows_prediction.png', target_size=(img_width,img_height))
    x = img_to_array(x)
    x = np.expand_dims(x, axis=0)
    array = model.predict(x)
    result = array[0]
    if result[0] > result[1]:
        if result[0] > 0.55:
            print("Predicted answer: Buy")
            answer = 'buy'
            print(result)
            print(array)
        else:
            print("Predicted answer: Not confident buy")
            answer = 'n/a'
            print(result)
        
    else:
        if result[1] > 0.55:
            print("Predicted answer: Sell")
            answer = 'sell'
            print(result)
        else:
            print("Predicted answer: Not confident sell")
            answer = 'n/a'
            print(result)

    return answer

In [ ]:
predict()